READING DATA FROM API AND WRITING TO DB

In [ ]:
import requests
import pandas as pd
from pymongo import MongoClient
from bson import json_util
import json

def fetch_data_from_api(api_url):
    try:
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()  
            return data
        else:
            print("Error: Unable to fetch data from API. Status code:", response.status_code)
            return None
    except requests.RequestException as e:
        print("Error: Unable to fetch data from API:", str(e))
        return None
api_url = "https://archive-api.open-meteo.com/v1/archive?latitude=53.3331&longitude=-6.2489&start_date=2024-03-30&end_date=2024-04-13&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,rain,snowfall,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m"
data = fetch_data_from_api(api_url)
if data:
    df = pd.DataFrame(data)
client = MongoClient("mongodb://%s:%s@127.0.0.1" % ("dap", "dap"))
db = client['test_database']
collection = db['weather']
records = json.loads(df.to_json(orient='records'))
collection.insert_many(records)
